In [ ]:
"""Irony classification using BiLSTM attention model with both fasttext and word2vec embeddings. Along with k-fold and hyperparameter tuning"""

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('gdrive/My Drive/anlp_project/train_preprocessed_df_fasttext.csv')
test_df = pd.read_csv('gdrive/My Drive/anlp_project/test_preprocessed_df_fasttext.csv')

In [4]:
import pickle

# Load the embeddings
with open('gdrive/My Drive/anlp_project/my_embeddings_fasttext.pkl', 'rb') as file:
    embeddings_dict = pickle.load(file)

# Example: To get the embedding of the word 'hello'
embedding_hello = embeddings_dict.get('hello', None)


In [5]:
embedding_hello.shape

(400,)

In [6]:
with open('gdrive/My Drive/anlp_project/my_emojis_fasttext.pkl', 'rb') as file:
    emojis_dict = pickle.load(file)

In [7]:
import torch
for emoji, embedding in emojis_dict.items():
    embedding_tensor = torch.tensor(embedding)
    padded_embedding = torch.cat([embedding_tensor, torch.zeros(100)])
    emojis_dict[emoji] = padded_embedding

In [8]:
hashtags_corpus = [key for key in embeddings_dict.keys() if key.startswith('#')]

In [9]:
len(hashtags_corpus)

2769

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class TweetDataset(Dataset):
    def __init__(self, dataframe, twitter_word2vec, emoji_vec, max_len):
        self.dataframe = dataframe
        self.twitter_word2vec = twitter_word2vec
        self.emoji_vec = emoji_vec
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        tweet_tokens = self.dataframe.iloc[idx]['tweet_tokens']

        # Generate embeddings for tokens
        embeddings = []
        for token in tweet_tokens:
            if token in self.twitter_word2vec:
                embeddings.append(torch.tensor(self.twitter_word2vec[token]))
            elif token in self.emoji_vec:
                embeddings.append(self.emoji_vec[token])
            else:
                # Handle unknown words or use a predefined UNK vector
                embeddings.append(torch.zeros(400))

        # Pad to max_len
        while len(embeddings) < self.max_len:
            embeddings.append(torch.zeros(400))

        # If the sequence is longer than max_len, truncate it
        embeddings = embeddings[:self.max_len]

        embeddings = torch.stack(embeddings)

        label = torch.tensor(self.dataframe.iloc[idx]['Label'])

        return embeddings, label


In [32]:
import torch
from torch.utils.data import Dataset, DataLoader

class TweetDataset_fasttext(Dataset):
    def __init__(self, dataframe, twitter_fasttext, emoji_vec, max_len):
        self.dataframe = dataframe
        self.twitter_fasttext = twitter_fasttext
        self.emoji_vec = emoji_vec
        self.max_len = max_len
        self.unk = 'UNK'
        self.hashtag = '#hashtag'

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        tweet_tokens = self.dataframe.iloc[idx]['tweet_tokens']

        # Generate embeddings for tokens
        embeddings = []
        for token in tweet_tokens:
            if token in self.twitter_fasttext:
                embeddings.append(torch.tensor(self.twitter_fasttext[token]))
            elif token in self.emoji_vec:
                embeddings.append(self.emoji_vec[token])
            elif token.startswith('#'):
                # Handles unknown hashtags
                embeddings.append(torch.tensor(self.twitter_fasttext[self.hashtag]))
            else:
                # Handle unknown words or use a predefined UNK vector
                embeddings.append(torch.tensor(self.twitter_fasttext[self.unk]))

        # Pad to max_len
        while len(embeddings) < self.max_len:
            embeddings.append(torch.zeros(400))

        # If the sequence is longer than max_len, truncate it
        embeddings = embeddings[:self.max_len]

        embeddings = torch.stack(embeddings)

        label = torch.tensor(self.dataframe.iloc[idx]['Label'])

        return embeddings, label


In [12]:
train_df.head()

,Tweet index,Label,Tweet text,text_prep,tweet_tokens
0,1,1,Sweet United Nations video. Just in time for C...,Sweet United Nations video. Just in time for C...,"['sweet', 'united', 'nations', 'video', 'punct..."
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,@mrdahl87 We are rumored to have talked to Erv...,"['we', 'are', 'rumored', 'to', 'have', 'talked..."
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,Hey there! Nice to see you Minnesota/ND Winter...,"['hey', 'there', '!', 'nice', 'to', 'see', 'yo..."
3,4,0,3 episodes left I'm dying over here,NUM episodes left I'm dying over here,"['num', 'episodes', 'left', ""i'm"", 'dying', 'o..."
4,5,1,I can't breathe! was chosen as the most notabl...,I can't breathe! was chosen as the most notabl...,"['i', ""can't"", 'breathe', '!', 'was', 'chosen'..."


In [33]:
# Determine max_len (e.g., based on the 95th percentile of tweet lengths)
max_len = int(train_df['tweet_tokens'].apply(len).quantile(0.95))

In [34]:
max_len

231

In [46]:
# Usage:
train_dataset = TweetDataset_fasttext(train_df, embeddings_dict, emojis_dict, max_len=max_len)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [47]:
test_dataset = TweetDataset_fasttext(test_df, embeddings_dict, emojis_dict, max_len=max_len)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [16]:
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob=0.1):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.fc = nn.Linear(2*hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)
        # We take the output from the final time step
        final_output = lstm_out[:, -1, :]
        out = self.fc(final_output)
        return out

# Example usage:
model = BiLSTM(input_dim=400, hidden_dim=256, num_layers=2, output_dim=1)  # Assuming binary classification for irony detection

In [59]:
import torch.nn.functional as F


class BiLSTM_Attention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob=0.0):
        super(BiLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.attention_linear = nn.Linear(2 * hidden_dim, 1)  # Attention scoring function
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)

        # Calculate attention scores
        att_scores = self.attention_linear(lstm_out)  # (batch_size, seq_len, 1)
        att_scores = att_scores.squeeze(-1)  # (batch_size, seq_len)
        att_weights = F.softmax(att_scores, dim=1).unsqueeze(2)  # (batch_size, seq_len, 1)

        # Calculate weighted sum of LSTM outputs
        weighted_sum = torch.sum(lstm_out * att_weights, dim=1)  # (batch_size, 2*hidden_dim)

        out = self.fc(weighted_sum)

        return out

# Example usage:
model = BiLSTM_Attention(input_dim=400, hidden_dim=256, num_layers=2, output_dim=1)  # Assuming binary classification for irony detection


In [60]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate of 0.001


In [61]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BiLSTM_Attention(
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, bidirectional=True)
  (attention_linear): Linear(in_features=512, out_features=1, bias=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [21]:
#pip install optuna

In [57]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score

In [21]:
def model_training_loop(model, optimizer, train_dataloader, test_dataloader, criterion):
  num_epochs = 15  # Example number of epochs
  best_test_f1 = 0
  epochs_without_improvement = 0
  patience = 3

  for epoch in range(num_epochs):

      model.train()
      running_loss = 0.0
      all_predictions = []
      all_labels = []
      all_probabilities = []

      for inputs, labels in train_dataloader:  # Assuming you have a DataLoader named train_loader
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()

          outputs = model(inputs).squeeze(1)  # get the model's predictions
          loss = criterion(outputs, labels.float())
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

          # Converting outputs to probabilities
          probabilities = torch.sigmoid(outputs)
          predictions = (probabilities > 0.5).float()
          all_predictions.extend(predictions.cpu().numpy())
          all_probabilities.extend(probabilities.cpu().detach().numpy())
          all_labels.extend(labels.cpu().numpy())

      epoch_loss = running_loss / len(train_dataloader)
      train_f1 = f1_score(all_labels, all_predictions)
      train_acc = accuracy_score(all_labels, all_predictions)
      precision = precision_score(all_labels, all_predictions)
      recall = recall_score(all_labels, all_predictions)
      #train_auc = roc_auc_score(all_labels, all_probabilities)

      model.eval()  # Set the model to evaluation mode
      test_loss = 0.0
      test_predictions = []
      test_labels = []
      test_probabilities = []

      with torch.no_grad():  # Disable gradient calculation
          for inputs, labels in test_dataloader:  # Assuming you have a DataLoader named test_dataloader
              inputs, labels = inputs.to(device), labels.to(device)

              outputs = model(inputs).squeeze(1)
              loss = criterion(outputs, labels.float())
              test_loss += loss.item()

              probabilities = torch.sigmoid(outputs)
              predictions = (probabilities > 0.5).float()
              test_predictions.extend(predictions.cpu().numpy())
              test_probabilities.extend(probabilities.cpu().numpy())
              test_labels.extend(labels.cpu().numpy())

      test_loss /= len(test_dataloader)
      test_f1 = f1_score(test_labels, test_predictions)
      test_acc = accuracy_score(test_labels, test_predictions)
      test_precision = precision_score(test_labels, test_predictions)
      test_recall = recall_score(test_labels, test_predictions)
      #test_auc = roc_auc_score(test_labels, test_probabilities)

      print(f"Epoch {epoch + 1}, train_f1: {train_f1}, train Accuracy: {train_acc}, test_f1: {test_f1}, test Accuracy: {test_acc}")

      # Early stopping logic
      if test_f1 > best_test_f1:
        best_test_f1 = test_f1
        epochs_without_improvement = 0
      else:
        epochs_without_improvement += 1

      if epochs_without_improvement >= patience:
        print(f"Early stopping due to no improvement in test F1 score and best f1 observed at epoch {epoch - patience -1}")
        break

      return best_test_f1




In [ ]:
model_training_loop(model, optimizer, train_dataloader, test_dataloader, criterion)

In [28]:
def objective(trial):
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
  dropout_prob = trial.suggest_uniform("dropout_prob", 0, 0.5)
  hidden_dim = trial.suggest_int("hidden_dim", 128, 512)  # You can adjust the range based on your needs
  num_layers = trial.suggest_int("num_layers", 1, 3)  # You can adjust the range based on your needs
  model = BiLSTM_Attention(input_dim=400, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=1, dropout_prob=dropout_prob).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCEWithLogitsLoss()
  print(lr, dropout_prob)
  #threshold = trial.suggest_uniform("threshold", 0.5, 1)
  best_test_f1 = model_training_loop(model, optimizer, train_dataloader, test_dataloader, criterion)
  return best_test_f1


In [29]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2023-10-01 11:20:55,728] A new study created in memory with name: no-name-7d014598-8b7d-4de0-b322-ffa6d066819d
<ipython-input-28-d02b52dcc8b9>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
<ipython-input-28-d02b52dcc8b9>:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_prob = trial.suggest_uniform("dropout_prob", 0, 0.5)


0.08000434414323418 0.09681175202299197


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-10-01 11:21:07,195] Trial 0 finished with value: 0.0 and parameters: {'lr': 0.08000434414323418, 'dropout_prob': 0.09681175202299197, 'hidden_dim': 332, 'num_layers': 2}. Best is trial 0 with value: 0.0.
<ipython-input-28-d02b52dcc8b9>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
<ipython-input-28-d02b52dcc8b9>:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_

Epoch 1, train_f1: 0.5031712473572938, train Accuracy: 0.5074665968037726, test_f1: 0.0, test Accuracy: 0.6033163265306123
7.31154157828076e-05 0.40542829408679304


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-10-01 11:21:19,605] Trial 1 finished with value: 0.0 and parameters: {'lr': 7.31154157828076e-05, 'dropout_prob': 0.40542829408679304, 'hidden_dim': 376, 'num_layers': 3}. Best is trial 0 with value: 0.0.
<ipython-input-28-d02b52dcc8b9>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
<ipython-input-28-d02b52dcc8b9>:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest

...

Epoch 1, train_f1: 0.5415266442630821, train Accuracy: 0.4996070212208541, test_f1: 0.5685557586837293, test Accuracy: 0.3979591836734694
0.0006965983179405797 0.2137081166285304


<ipython-input-28-d02b52dcc8b9>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
<ipython-input-28-d02b52dcc8b9>:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_prob = trial.suggest_uniform("dropout_prob", 0, 0.5)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2137081166285304 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

Epoch 1, train_f1: 0.4830230010952903, train Accuracy: 0.505370709981661, test_f1: 0.0, test Accuracy: 0.6033163265306123
0.0005239091909184562 0.19005325187514965


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-10-01 11:29:33,574] Trial 49 finished with value: 0.0 and parameters: {'lr': 0.0005239091909184562, 'dropout_prob': 0.19005325187514965, 'hidden_dim': 433, 'num_layers': 3}. Best is trial 11 with value: 0.5789473684210527.


Epoch 1, train_f1: 0.5080687142113482, train Accuracy: 0.5048467382761331, test_f1: 0.0, test Accuracy: 0.6033163265306123


In [30]:
completed_trials = study.trials_dataframe()

In [31]:
completed_trials

,number,value,datetime_start,datetime_complete,duration,params_dropout_prob,params_hidden_dim,params_lr,params_num_layers,state
0,0,0.000000,2023-10-01 11:20:55.731875,2023-10-01 11:21:07.195325,0 days 00:00:11.463450,0.096812,332,0.080004,2,COMPLETE
1,1,0.000000,2023-10-01 11:21:07.196488,2023-10-01 11:21:19.605359,0 days 00:00:12.408871,0.405428,376,0.000073,3,COMPLETE
2,2,0.000000,2023-10-01 11:21:19.606448,2023-10-01 11:21:29.406349,0 days 00:00:09.799901,0.111952,230,0.001267,1,COMPLETE
3,3,0.000000,2023-10-01 11:21:29.407529,2023-10-01 11:21:40.796614,0 days 00:00:11.389085,0.236342,257,0.000025,2,COMPLETE
4,4,0.040462,2023-10-01 11:21:40.797748,2023-10-01 11:21:50.157985,0 days 00:00:09.360237,0.270611,180,0.000704,1,COMPLETE
5,5,0.000000,2023-10-01 11:21:50.159188,2023-10-01 11:22:02.510876,0 days 00:00:12.351688,0.038259,344,0.069033,3,COMPLETE
6,6,0.568037,2023-10-01 11:22:02.512804,2023-10-01 11:22:12.328257,0 days 00:00:09.815453,0.165811,361,0.000865,1,COMPLETE
7,7,0.000000,2023-10-01 11:22:12.329491,2023-10-01 11:22:25.400230,0 days 00:00:13.070739,0.407643,402,0.000063,3,COMPLETE
8,8,0.000000,2023-10-01 11:22:25.401353,2023-10-01 11:22:38.718939,0 days 00:00:13.317586,0.222405,484,0.000231,3,COMPLETE
9,9,0.000000,2023-10-01 11:22:38.720190,2023-10-01 11:22:50.294501,0 days 00:00:11.574311,0.009940,390,0.006875,2,COMPLETE


In [62]:
model = model.to(device)

num_epochs = 13  # Example number of epochs

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0
    all_predictions = []
    all_labels = []
    all_probabilities = []

    for inputs, labels in train_dataloader:  # Assuming you have a DataLoader named train_loader
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs).squeeze(1)  # get the model's predictions
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Converting outputs to probabilities
        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > 0.5).float()
        all_predictions.extend(predictions.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().detach().numpy())
        all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(train_dataloader)
    f1 = f1_score(all_labels, all_predictions)
    acc = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    auc = roc_auc_score(all_labels, all_probabilities)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Loss: {epoch_loss:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"AUC: {auc:.4f}")
    print('-' * 30)


Epoch 1/13
Loss: 0.6945
F1 Score: 0.4952
Accuracy: 0.5038
Precision: 0.5019
Recall: 0.4887
AUC: 0.5007
------------------------------
Epoch 2/13
Loss: 0.6917
F1 Score: 0.5575
Accuracy: 0.5067
Precision: 0.5038
Recall: 0.6239
AUC: 0.5160
------------------------------
Epoch 3/13
Loss: 0.6810
F1 Score: 0.6104
Accuracy: 0.5476
Precision: 0.5344
Recall: 0.7117
AUC: 0.5595
------------------------------
Epoch 4/13
Loss: 0.6839
F1 Score: 0.5652
Accuracy: 0.5428
Precision: 0.5369
Recall: 0.5965
AUC: 0.5657
------------------------------
Epoch 5/13
Loss: 0.6650
F1 Score: 0.6436
Accuracy: 0.5921
Precision: 0.5697
Recall: 0.7396
AUC: 0.6105
------------------------------
Epoch 6/13
Loss: 0.6611
F1 Score: 0.6491
Accuracy: 0.5889
Precision: 0.5646
Recall: 0.7633
AUC: 0.6217
------------------------------
Epoch 7/13
Loss: 0.6547
F1 Score: 0.6480
Accuracy: 0.6041
Precision: 0.5815
Recall: 0.7317
AUC: 0.6381
------------------------------
Epoch 8/13
Loss: 0.6490
F1 Score: 0.6553
Accuracy: 0.6183
Prec

In [24]:
state_dict = torch.load('gdrive/My Drive/anlp_project/bilstm_attention_model.pth')

In [35]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [37]:
model = model.to(device)

In [63]:
# Evaluate on test data
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_predictions = []
test_labels = []
test_probabilities = []

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_dataloader:  # Assuming you have a DataLoader named test_dataloader
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels.float())
        test_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > 0.5).float()
        test_predictions.extend(predictions.cpu().numpy())
        test_probabilities.extend(probabilities.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_loss /= len(test_dataloader)
test_f1 = f1_score(test_labels, test_predictions)
test_acc = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions)
test_recall = recall_score(test_labels, test_predictions)
test_auc = roc_auc_score(test_labels, test_probabilities)

print("Test Evaluation:")
print(f"Loss: {test_loss:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Accuracy: {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"AUC: {test_auc:.4f}")
print('-' * 30)


Test Evaluation:
Loss: 0.6499
F1 Score: 0.5510
Accuracy: 0.6071
Precision: 0.5040
Recall: 0.6077
AUC: 0.6638
------------------------------


In [64]:
import numpy as np
best_threshold = 0.5  # Default threshold
best_f1 = 0.0

# Iterate over potential thresholds
thresholds = np.linspace(0, 1, 100)
for threshold in thresholds:
    predictions = (np.array(test_probabilities) > threshold).astype(int)
    f1 = f1_score(test_labels, predictions)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

test_predictions = (np.array(test_probabilities) > best_threshold).astype(int)
test_acc = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions)
test_recall = recall_score(test_labels, test_predictions)
test_auc = roc_auc_score(test_labels, test_probabilities)

print("Test Evaluation:")
print(f"Loss: {test_loss:.4f}")
print(f"Best Threshold: {best_threshold:.4f}")
print(f"F1 Score: {best_f1:.4f}")
print(f"Accuracy: {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"AUC: {test_auc:.4f}")
print('-' * 30)

Test Evaluation:
Loss: 0.6499
Best Threshold: 0.1818
F1 Score: 0.6092
Accuracy: 0.5255
Precision: 0.4524
Recall: 0.9325
AUC: 0.6638
------------------------------


In [65]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'gdrive/My Drive/anlp_project/bilstm_attention_model_fasttext.pth')

In [31]:
# K Fold training

In [37]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Constants
num_epochs = 20
k_folds = 5

# K-fold Cross Validation model evaluation
#kfold = KFold(n_splits=k_folds, shuffle=True)
kfold = StratifiedKFold(n_splits=k_folds, shuffle=True)

results = {}  # To store results of each fold

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_df, train_df['Label'])):  # Note that we pass train_df['Label'] as the second argument

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

    train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_subsampler)
    val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_subsampler)

    print(f'FOLD {fold + 1}')
    print('-' * 40)

    # Assuming `model` and optimizer are fresh and initialized
    # Consider re-initializing model and optimizer at the start of each fold
    model = BiLSTM_Attention(input_dim=400, hidden_dim=256, num_layers=2, output_dim=1)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate of 0.001

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_predictions = []
        all_labels = []
        all_probabilities = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            probabilities = torch.sigmoid(outputs)
            predictions = (probabilities > 0.5).float()
            all_predictions.extend(predictions.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().detach().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Logging for each epoch
        epoch_loss = running_loss / len(train_loader)
        f1 = f1_score(all_labels, all_predictions)
        acc = accuracy_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions)
        recall = recall_score(all_labels, all_predictions)
        auc = roc_auc_score(all_labels, all_probabilities)

        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Train F1: {f1:.4f}, Train Acc: {acc:.4f}, Train AUC: {auc:.4f} ")

        # Validation loss
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_labels = []
        val_probabilities = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs).squeeze(1)
                loss = criterion(outputs, labels.float())
                val_loss += loss.item()

                probabilities = torch.sigmoid(outputs)
                predictions = (probabilities > 0.5).float()
                val_predictions.extend(predictions.cpu().numpy())
                val_probabilities.extend(probabilities.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_f1 = f1_score(val_labels, val_predictions)
        val_acc = accuracy_score(val_labels, val_predictions)
        val_auc = roc_auc_score(val_labels, val_probabilities)
        results[fold] = val_acc  # Store the accuracy for this fold

        print(f"Validation Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}")
        print('-' * 30)

# Print final results
print(f"K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS")
print('-' * 40)
print(f"Fold | Accuracy")
print('-' * 40)
for key, value in results.items():
    print(f"{key + 1} | {value:.4f}")


FOLD 1
----------------------------------------
Epoch 1/20 - Train Loss: 0.6947, Train F1: 0.4865, Train Acc: 0.4884, Train AUC: 0.4851 
Validation Loss: 0.6932, Val F1: 0.6655, Val Acc: 0.4987, Val AUC: 0.5411
------------------------------
Epoch 2/20 - Train Loss: 0.6931, Train F1: 0.5071, Train Acc: 0.5123, Train AUC: 0.5153 
Validation Loss: 0.6914, Val F1: 0.6750, Val Acc: 0.5223, Val AUC: 0.6175
------------------------------
Epoch 3/20 - Train Loss: 0.6906, Train F1: 0.5748, Train Acc: 0.5179, Train AUC: 0.5220 
Validation Loss: 0.6902, Val F1: 0.6797, Val Acc: 0.5484, Val AUC: 0.5651
------------------------------
Epoch 4/20 - Train Loss: 0.6868, Train F1: 0.6088, Train Acc: 0.5319, Train AUC: 0.5339 
Validation Loss: 0.7094, Val F1: 0.5529, Val Acc: 0.5131, Val AUC: 0.5636
------------------------------
Epoch 5/20 - Train Loss: 0.6843, Train F1: 0.6041, Train Acc: 0.5300, Train AUC: 0.5289 
Validation Loss: 0.6769, Val F1: 0.5486, Val Acc: 0.5262, Val AUC: 0.5768
-------------